In [1]:
import time
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
import jieba
import re
from useful_tools import * # 导入自编函数

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\xsong\AppData\Local\Temp\jieba.cache
Loading model cost 1.278 seconds.
Prefix dict has been built succesfully.


In [2]:
train = pd.read_csv("../data/TrainSet.csv")
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y = le.fit_transform(train['class'])

mapping = dict(zip(le.classes_, le.transform(le.classes_)))
mapping
cutWords_series = train['content'].apply(lambda x: get_cutword(x)) # 得到的是pandas series
unigram = get_vectorize(cutWords_series)

In [3]:
from sklearn.model_selection import GridSearchCV
def param_tune(classifier,x_train, y_train, param_grid, cv = 10):
    '''
    调参函数，输入参数范围，输出运行时间、最优参数
    '''
    startTime = time.time()
    gsearch = GridSearchCV(classifier,scoring = 'accuracy',
                           param_grid = param_grid,
                           iid = True, cv = cv)
    gsearch.fit(x_train, y_train)
    print('最高准确率为%.3f' %(gsearch.best_score_))
    totaltime = time.time() - startTime
    print('共花费%.2f秒' %(totaltime))
    print(gsearch.best_params_)
    return gsearch.best_estimator_

In [4]:
param_test1 = {
 'max_depth': range(2,5,1),
 'min_child_weight': range(2,5,1)
}

param_test2 = { 
 'max_depth': range(2,3,1),
 'min_child_weight': range(2,3,1),
 'subsample':[ i / 10.0 for i in range(8,10)],
 'colsample_bytree':[ i / 10.0 for i in range(6,7)]
              }

In [5]:
from xgboost import XGBClassifier
xgb = XGBClassifier( n_estimators=100)
param_tune(xgb, unigram, y, param_test2 ,10)

最高准确率为0.745
共花费28.80秒
{'colsample_bytree': 0.6, 'max_depth': 2, 'min_child_weight': 2, 'subsample': 0.9}


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.6, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=2,
              min_child_weight=2, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='multi:softprob', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=0.9, verbosity=1)

调参结果：

```
最高准确率为0.73
共花费245.64秒
{'max_depth': 3, 'min_child_weight': 4}

最高准确率为0.74
共花费425.80秒
{'colsample_bytree': 0.4, 'subsample': 0.5}

最高准确率为0.758
共花费1323.96秒
{'colsample_bytree': 0.5, 'max_depth': 3, 'min_child_weight': 2, 'subsample': 0.5}

最高准确率为0.767
共花费212.76秒
{'colsample_bytree': 0.6, 'max_depth': 2, 'min_child_weight': 2, 'subsample': 0.8}
```